In [337]:
import pandas as pd
import numpy as np
import h5py
import matplotlib
import math

#small_data = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/CS170_Small_Data__91.txt'
small_data = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/test/Sue/CS170_Small_Data__96.txt'
small_data_df = pd.read_csv(small_data,
                            sep ='  ',
                            engine = 'python',
                            header = None)

large_data = '/Users/brandonchea/Desktop/Fall_2022/CS170/Project2/Datasets/CS170_Large_Data__103.txt'
large_data_df = pd.read_csv(large_data, 
                         sep='  ',
                         engine = 'python',
                         header=None)




#we take the dataframe, we are provided a set of features
#we wish to add a feature
#we do this by going through the data frame, add the feature_to_add to the current set
#then we remove the features we don't want
#def leave_one_out_cross_validation(data_df,current_set,feature_to_add):

def stub(data_df,current_set,feature_to_add):
    return np.random.rand()

def dist(a,b):
    diff = a[1:] - b[1:]
    return(np.dot(diff,diff))

#small_data_df = small_data_df.drop([2,4,5],axis=1)
def nn(data_df,current_set,feature_to_add): #this is correct
    current_set.append(feature_to_add)
    
    drops = []
    x = 1
    for x in data_df.iloc[:,1:]:
        if x not in current_set:
            drops.append(x)
            
        
    data_df.loc[:,drops] = 0
    number_correctly_classified = 0
    
    for i in range(0,len(data_df)):
        object_to_classify = data_df.iloc[i]
        #object_to_classify = data_df[i]
        label_object_to_classify = data_df.iloc[i][0]
        
        
        nearest_neighbor_distance = float('inf')
        nearest_neighbor_location = float('inf')
        
        for k in range(0,len(data_df)):
            if k!=i:
                distance = dist(object_to_classify,data_df.iloc[k])

                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = k
                    nearest_neighbor_label = data_df.iloc[nearest_neighbor_location][0]
                                   
        if label_object_to_classify == nearest_neighbor_label:
            number_correctly_classified = number_correctly_classified +1 
        
    accuracy = number_correctly_classified / len(data_df)
    #print(accuracy)
    return accuracy

'''   
current_set = [1,2]
feature_to_add = 3
nn(small_data_df,current_set,feature_to_add)'''    

def leave_one_out_cross_validation(data_df,current_set,feature_to_add):
    current_set.append(feature_to_add)
    not_in_set = []
    for k in current_set:
        if k not in current_set:
            data_df.iloc[:,k] = 0 #set all column values not within current set to 0
            #data_df = data_df.drop([k],axis=1)

    accuracy = nn(data_df)
    return accuracy


#current_set = [1,3,6]
'''for k in current_set:
        if k not in current_set:
            #data_df[k].values[:] = 0 #set all column values not within current set to 0
            small_data_df = small_data_df.drop([k],axis=1)'''
    
#small_data_df = small_data_df.drop([2,4,5],axis=1)
#large_data_df = large_data_df.drop([])
#small_data_df = 
#small_data_df.iloc[:,2] = 0
#small_data_df.iloc[:,4] = 0
#small_data_df.iloc[:,5] = 0
#for col in small_data_df.columns
 
#small_data_df.head()
#knn(small_data_df)       
#small_data_df.head()

#nn(small_data_df)

'for k in current_set:\n        if k not in current_set:\n            #data_df[k].values[:] = 0 #set all column values not within current set to 0\n            small_data_df = small_data_df.drop([k],axis=1)'

In [343]:
def feature_search_demo(data_df):
    current_set_of_features = []
    i = 1
    #for i in range(len(small_data_df)-490):
    for i in data_df.iloc[:,1:]:
        print("On the ",i," level of the search tree")
        feature_to_add = []
        best_so_far_acc = 0
        #k = 1
        for k in data_df.iloc[:,1:]:
            if k not in current_set_of_features:
                print("Considering adding feature ",k)
                accuracy = nn(data_df,current_set_of_features,k)
                #print("We have accuracy",accuracy)
                if (accuracy > best_so_far_acc):
                    best_so_far_acc = accuracy
                    feature_to_add = k
            current_set_of_features.append(feature_to_add)
        print("On level ",i," I added feature",feature_to_add," to current set with accuracy",best_so_far_acc)
        print(current_set_of_features)
        
feature_search_demo(small_data_df)

On the  1  level of the search tree
Considering adding feature  1


/var/folders/df/_z0dpj9528j3trt78v6wyhc00000gn/T/ipykernel_55864/305185325.py:33: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  diff = a[1:] - b[1:]


Considering adding feature  2


/var/folders/df/_z0dpj9528j3trt78v6wyhc00000gn/T/ipykernel_55864/305185325.py:33: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  diff = a[1:] - b[1:]


Considering adding feature  3


...

Considering adding feature  5


/var/folders/df/_z0dpj9528j3trt78v6wyhc00000gn/T/ipykernel_55864/305185325.py:33: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  diff = a[1:] - b[1:]


Considering adding feature  6


/var/folders/df/_z0dpj9528j3trt78v6wyhc00000gn/T/ipykernel_55864/305185325.py:33: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  diff = a[1:] - b[1:]


On level  1  I added feature 1  to current set with accuracy 0.874
[1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1]
On the  2  level of the search tree
On level  2  I added feature []  to current set with accuracy 0
[1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, [], [], [], [], [], []]
On the  3  level of the search tree
On level  3  I added feature []  to current set with accuracy 0
[1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, [], [], [], [], [], [], [], [], [], [], [], []]
On the  4  level of the search tree
On level  4  I added feature []  to current set with accuracy 0
[1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
On the  5  level of the search tree
On level  5  I added feature []  to current set with accuracy 0
[1, 1, 2, 1, 3, 1, 4, 1, 5, 1, 6, 1, [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
On the  6  level of the search tree
On level  6  I added feature []  to current set with accuracy 0
[1, 1, 2

In [254]:
def knn(data_df):
#def leave_one_out_cross_validation(data_df,current_set,feature_to_add):
    number_correctly_classified = 0
    
    for i in range(1,len(data_df)):
        object_to_classify = data_df.iloc[i]
        label_object_to_classify = data_df.iloc[i][0]
        
        nearest_neighbor_distance = float('inf')
        nearest_neighbor_location = float('inf')
        
        for k in range(1,len(data_df)):
            if k!=i:
                distance = math.sqrt(sum(object_to_classify - data_df.iloc[k])**2)
                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = k
                    nearest_neighbor_label = data_df.iloc[nearest_neighbor_location][0]
                #print("Ask if ", i, " is neighest neighbor with",k)
        
        #print("Looping over i, at the ",i,"location")
        #print("The ",i,"th object is in class",label_object_to_classify)
        if label_object_to_classify == nearest_neighbor_label:
            number_correctly_classified +=1
        #print("Object ",i, " is class ", label_object_to_classify)
        #print("It's nearest neighbor is ", nearest_neighbor_location, " which is in class ", nearest_neighbor_label)
    accuracy = number_correctly_classified / len(data_df)
    #print(accuracy)
    return accuracy
        
knn(small_data_df)

KeyboardInterrupt: 